# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn

import torch.nn.functional as F
import numpy as np

import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader
import glob

from torchvision import datasets, transforms
from src.models import training, linear_vanilla_vae, denoising_vanilla_vae, skip_vae

from src.features import utils
from src.features import dataLoader as MyDataLoader
import pathlib
import PIL
from torchvision.transforms import ToTensor, ToPILImage

from imageaugment import augment
import PIL.Image
import pathlib
import json

# Load Full Samples of Templates

In [5]:
x_train_tensor, x_val_tensor, mean, std = utils.load_full_image_tensors(pathlib.Path("/home/fahad/master_thesis/data/simulated_noisy_templates_with_gaussian_noise/"))

In [6]:
print(x_train_tensor.shape)
print(x_val_tensor.shape)

torch.Size([5000, 1, 1536, 1024])
torch.Size([800, 1, 1536, 1024])


# Construct Model

In [7]:
z_dim=256
in_channels = 1
init_filters = 32
n_filters_per_depth=2
n_depth=2
device = torch.device("cuda:0")

latent space dim 8388608


In [8]:
vae = denoising_vanilla_vae.VAE(z_dim=z_dim, in_channels=in_channels,init_filters = init_filters, n_filters_per_depth=n_filters_per_depth,n_depth=n_depth)

In [9]:
batch_size=5

train_dataset = MyDataLoader.MyDataset(x_train_tensor,x_train_tensor)
val_dataset = MyDataLoader.MyDataset(x_val_tensor,x_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

directory_path = "/home/fahad/master_thesis/vanilla_vae/models/linear_vae/"
n_epochs = 300
net = vae
device = device

lr=0.001
val_loss_patience = 100
data_mean = mean.item()
data_std = std.item()
gaussian_noise_std = 1.0

model_name = "template-"

# Training

In [ ]:
trainHist, reconHistory, klHist, valHist = training.trainNetwork(net=net, train_loader=train_loader, 
                                                                 val_loader=val_loader,
                                                                 device=device,directory_path=directory_path,
                                                                 model_name=model_name,
                                                                 n_epochs=n_epochs, batch_size=batch_size,lr=lr,
                                                                 val_loss_patience = val_loss_patience,
                                                                 kl_annealing = True,
                                                                 kl_start = 0, 
                                                                 kl_annealtime = 3,
                                                                 data_mean =data_mean,data_std=data_std, 
                                                                 gaussian_noise_std = gaussian_noise_std)

# Skip VAE

In [12]:
seed = 100
USE_CUDA = torch.cuda.is_available()
net = skip_vae.VAE(
    conv_layers=32,
    z_dimension=64,
    pool_kernel_size=2,
    conv_kernel_size=4,
    input_channels=1,
    height=1536,
    width=1024,
    hidden_dim=128,
    use_cuda=USE_CUDA,
    use_skip_connections=True
)